# INFO 408 Lab Exercise 12: Apache Spark continued

This lab exercise continues the exploration of Apache Spark started in Lab 11. For convenience, the instructions for Lab 10 are included [at the end](#INFO408-Lab-Exercise-11:-Apache-Spark) so that you can refer to the information given there, and continue working on that lab exercise if you did not finish it last week.

## IMPORTANT

**This notebook is for use only with the computers in lab OBS 327.** It is configured only for that location and will not work correctly elsewhere. If you want to use Docker (with or without VirtualBox) on your own computer, download `lab12-docker.ipynb` instead.

## Recipients in common

The following Spark query will recreate the three data frames that you used to explore the Enron dataset in Lab 11, except this time the first line only keeps a sample of 10% of the messages (the `False` in the first argument of the `sample` method means that sampling with replacement is **not** used). The final data frame, `no_self_mails_sender_recipient_pairs` contains a row for each `sender`/`recipient` pair in the 10% sample of the Enron dataset. A `count` column records how many emails were sent from the sender to the recipient.

In [ ]:
messages = spark.read.json("/mnt/t-drive/InfoSci/INFO408/Spark/messages.json").sample(False, 0.1)
messages.createOrReplaceTempView("messages") # This is needed before doing an SQL query
sender_recipient_pairs = spark.sql(
    "select messages.headers.From as sender, recipient, count(*) as count "
    "from messages lateral view explode(split(messages.headers.To, '[,\\\\s]+')) tolist as recipient "
    "group by sender, recipient order by count desc"
)
sender_recipient_pairs.createOrReplaceTempView("sender_recipient_pairs")
no_self_mails_sender_recipient_pairs = spark.sql(
    "select sender, recipient, count from sender_recipient_pairs "
     "where sender != recipient"
)
print "Schema:"
sender_recipient_pairs.printSchema()
no_self_mails_sender_recipient_pairs.createOrReplaceTempView("no_self_mails_sender_recipient_pairs")

**Question:** Can you generate an RDD in which each element contains a unique pair of users (senders) and a list of the email addresses (recipients) that they have both sent messages to? In other words, for each pair of senders, which recipients do they have in common?

See [this article](http://stevekrenzel.com/finding-friends-with-mapreduce) about a map-reduce algorithm for finding *friends in common* (scroll down past the word count example). This is the same example that was discussed in Lecture 11.

Here is a Spark implementation of the friends in common algorithm (with an error corrected):

In [ ]:
from functools import reduce

friends = sc.parallelize([
    ('a',['b','c','d']),
    ('b',['a','c','d','e']),
    ('c',['a','b','d','e']),
    ('d',['a','b','c','e']),
    ('e',['b','c','d']),
    ('z', ['e'])
])
mapped = friends.flatMap(lambda pair: [(tuple(sorted([pair[0], friend])), set(pair[1])) for friend in pair[1]])
grouped = mapped.groupByKey().mapValues(list)
grouped_no_singletons = grouped.mapValues(lambda l: l if len(l) == 2 else l + [set()])
intersected = grouped_no_singletons.map(lambda pair: (pair[0], reduce(set.intersection, pair[1])))
intersected.collect()

Now, you just need to create a friends RDD from the messages data set, with a similar structure to the input to the `sc.parallelize` call above: each entry will be a pair of the form *(sender, list of all recipients that sender has sent messages to)*. Then you can use the code in the cell above to answer the question.

You can start by converting the `no_self_mails_sender_recipient_pairs` data frame to an RDD by using the `rdd` attribute:

In [ ]:
rdd = no_self_mails_sender_recipient_pairs.rdd

Remember that this RDD will contain `Row` objects (see Lab 11, Exercise 2).

You might like to immediately replace each row in the RDD with a tuple:

In [ ]:
tuple_rdd = rdd.map(lambda row: (row.sender, row.recipient))

Now you can call `tuple_rdd.groupByKey().mapValues(list)` to get the RDD you need as input to the friends in common algorithm. *Technical detail*: `mapValues(list)` is needed because the RDD’s `groupByKey` method creates groupings that are not lists; rather they are Python “iterables”. `mapValues(list)` turns the iterables into lists.

In [ ]:
# Enter your code here


## Graphframes

Now we will use a third-party `graphframes` library that will let us view the data as a graph or network with a directed edge between each sender and recipient. First we must create data frames representing the vertices and edges of the graph, and these must have specific column names (`id` for the vertices, and `src` and `dst` for the edges). Then we can search for pairs of users (a, b) that satisfy the pattern `(a)-[]->(b); !(b)-[]->(a)`. This means that there is an edge (i.e., emails were sent) from `a` to `b`, but not from `b` to `a`.

This query may take some time, so you might like to read about the [graphframes library](https://graphframes.github.io/graphframes/docs/_site/user-guide.html) while you wait.

In [ ]:
from graphframes import *

no_self_mails_sender_recipient_pairs.createOrReplaceTempView("no_self_mails_sender_recipient_pairs")
vertices = spark.sql(
    "select sender as id from no_self_mails_sender_recipient_pairs "
    "union "
    "select recipient from no_self_mails_sender_recipient_pairs")
edges = no_self_mails_sender_recipient_pairs.toDF('src','dst','count')
gf = GraphFrame(vertices, edges)
gf.find('(a)-[]->(b); !(b)-[]->(a)').take(5)

Can you think of other interesting queries to try using `find` and graph motifs? 

Other useful `GraphFrame` attributes and methods (besides `find`) include `inDegrees` (for each user, how many other users have mailed the user) and `outDegrees` (for each user, how many other users have been emailed by that user), `pageRank(tol=...)`, and `stronglyConnectedComponents(maxIter)`  (where `maxIter` is the maximum number of iterations to run). Note that `inDegrees` and `outDegrees` are graphframe *attributes*: no brackets are added after these.

`pageRank(tol=...)` is the original web page ranking algorithm used by Google. In the context of the Enron emails, the results could be interpreted as the probability for each user that a piece of gossip randomly arriving at one of the users will be passed on through a chain of emails to that specified user (See [Wikipedia](https://en.wikipedia.org/wiki/PageRank)). The `tol` parameter specifies how much error can be tolerated (e.g., 0.01). The result of `pageRank` is another `GraphFrame`. You can find the page rank for the first vertex using `pageRankResult.vertices.take(1)`.

See [Wikipedia](https://en.wikipedia.org/wiki/Strongly_connected_component) for a definition of strongly connected components. Running `stronglyConnectedComponents(maxIter)` on the Enron data (even with our 10% sample) may take a long time, depending on what value you provide for the `maxIter` argument. Also, rather than printing the components, it might be best just to count them, using `.count()`.


In [ ]:
# Try out other graphframe queries here. Insert more cells if you need to.


Finally, note that we can use `graphframes` to solve the “friends in common” problem. Here we use a small test `GraphFrame` so the computation is quick.

In [ ]:
import pyspark.sql.functions as F

# Vertex DataFrame
v = sqlContext.createDataFrame([
    ("a", "Alice", 34),
    ("b", "Bob", 36),
    ("c", "Charlie", 30),
    ("d", "David", 29),
    ("e", "Esther", 32),
    ("f", "Frances", 36)
], ["id", "name", "age"])

# Edge DataFrame
e = sqlContext.createDataFrame([
    ("a", "b", "friend"),
    ("a", "f", "friend"),
    ("b", "c", "friend"),
    ("c", "b", "friend"),
    ("c", "f", "friend"),
    ("f", "c", "friend"),
    ("e", "f", "friend"),
    ("e", "d", "friend"),
    ("d", "a", "friend")
], ["src", "dst", "relationship"])

gf2 = GraphFrame(v,e)
gf2.find('(x)-[]->(z); (y)-[]->(z)').filter("x != y").groupBy("x", "y").agg(F.collect_set("z")).take(5)

# INFO408 Lab Exercise 11: Apache Spark

The aim of this lab exercise is to explore the features of Apache Spark for processing large data sets.

This Jupyter notebook is designed to interact with Apache Spark. It uses Python 3.6.

## A quick reminder of how to use Jupyter notebooks

Type `Control-Enter` in a cell to run the code. If you see `[*]` to the left of a cell being run, the code is still running. (The Spark commands in Exercise 2 analyse half a million email messages, and will therefore take some time to run.)

Use the **Insert** menu to add new cells.

Save your progress using **File** ▷ **Save and Checkpoint**.

When you have finished this lab, you can create a (non-interactive) copy of this notebook in HTML format using **File** ▷ **Download as…**

To exit, select **File** ▷ **Close and Halt**. Do not worry if error messages are displayed in the terminal window from which you started the cluster. You can then type `Control-C` followed by `docker-compose down` to shut down the cluster.

## Useful links

Note that this exercise uses version 2.4.3 of Spark, not version 3.

 * [Spark RDD programming guide](https://spark.apache.org/docs/2.4.3/rdd-programming-guide.html)
 * [Python API documentation for the RDD class](https://spark.apache.org/docs/2.4.3/api/python/pyspark.html#pyspark.RDD)
 * [Spark SQL, DataFrames and Datasets Guide](https://spark.apache.org/docs/2.4.3/sql-programming-guide.html)
 * [Python API documentation for the DataFrame class](https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.DataFrame)
 * [Python standard library (standard datatypes and functions)](https://docs.python.org/3.6/library/index.html)

## Exercise 1: Exploring the Shakespeare data set

In this exercise you will use a data file (`shakespeare.txt`) containing the complete works of William Shakespeare (plays and poems). This comprises about 8.8 MB of semi-structured text.

To query the data using Spark, run the code in each of the following cells by clicking on a cell and then typing `Control-Enter` (`⌘-Enter` on a Mac). Then look at the output, and try to understand what the code is doing.

The variable `sc` is predefined by PySpark to hold a reference to a *Spark Context* object. This is configured to run Spark in the cluster that you created by running `docker-compose up`. Each node in the cluster is available to store parts of the data and to run computations on them. You can also submit jobs to the cluster via `pyspark` (the Python Spark interpreter) running inside this notebook.

In [ ]:
sc

Now load the data.

In [ ]:
lines = sc.textFile("/mnt/t-drive/InfoSci/INFO408/Spark/shakespeare.txt")
lines

The variable `lines` now holds a reference to an object of class RDD: a *resilient distributed dataset*. The dataset is automatically partitioned with each partition held in memory in one of the executor processes.

Now let us see how many lines of data were read.

In [ ]:
lines.count()

The cell below calls the `take(n)` method. This retrieves the first *n* data items from the RDD and returns them as a Python list, so the list must be small enough to fit into memory.

In [ ]:
lines.take(10)

Note: `\t` represents a tab character.

Note that each line begins with a *key*: the name of the work (a play or poem) followed by `@` and a byte offset from the start of the work.

Now let us see how the data has been partitioned by Spark.

In [ ]:
lines.getNumPartitions()

The result of `getNumPartitions()` shows that the data has been broken up into separate partitions by Spark. The number of partitions would probably be larger if you were running this code on a “real” Spark cluster.

Next, to see the range of methods (functions associated with a class) that are available to be called on an RDD object, position the cursor after the `.` in the cell below and press the `Tab` key (you may need to delete and re-enter the `.` for this to work, and it may be slow to respond). Use the `Page Down` key to see all the methods, and `Esc` to close the list. If you want to read what these methods do, see the [documentation for the DataFrame class](https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.DataFrame).

Note: You do not need to *run* the cell below, but if you do, you will get an error message unless you complete the line to call an RDD class method with valid arguments.

In [ ]:
lines.

Now let us get rid of the key at the start of each line in the data set. Compare the result with that from `lines.take(10)` above. The `map` method takes a function as an argument and applies it to each item in the data set (in this case, a line of text). A new RDD is returned with each item replaced by its mapped value.

Here we do the mapping using a lambda function (see Lab 4). This takes one string argument (`line`), splits it at the first (`maxsplit=1`) tab character (`sep="\t"`) and returns the second substring (`[1]`, because Python indexes lists from 0). As you can see from the result this is not perfect, as there are still some lines that start with a tab character, but it is good enough for our purposes. (If you are interested, investigate Python’s `re` module that provides advanced pattern matching and substitution using regular expressions.)

In [ ]:
import re
lines2 = lines.map(lambda line: line.split("\t", 1)[1])
lines2.take(10)

Python’s `lambda` syntax is very useful for passing short unnamed functions to functions like map and reduce. The general syntax is:

```python
  lambda arg1, ..., argn: SomeExpressionUsingTheArguments
```

The expression is evaluated and its result is returned when the function is called.

Now we will create a new RDD containing the description of the plays’ scenes. The `collect` method retrieves all data from an RDD and returns it as a Python list. You should therefore `collect` if the RDD is small enough to fit into memory.

In [ ]:
scenes = lines2.filter(lambda line: "SCENE " in line)
scenes.collect()

Next, we generate a sorted RDD of pairs (count, word) for each word in the data set. `sortByKey(ascending=False)` means that the result will be sorted in *descending* order, i.e., most frequently used words first. If you want to see the *least* frequently used words, change `False` to `True`.

In [ ]:
punctuation = ".?,:;-"
remove_punct_mapping = dict.fromkeys(map(ord, punctuation)) # Do not worry about what this means
wordcounts = \
    lines2.map(lambda x: x.translate(remove_punct_mapping).lower()) \
    .flatMap(lambda x: x.split()) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda x,y: x+y) \
    .map(lambda x: (x[1],x[0])) \
    .sortByKey(ascending=False)
wordcounts.take(10)

The backslash characters at the end of some of the lines above tell Python that the current line of code continues on the next line. This is useful when we have to chain together a long series of method calls.

An alternative approach to these long call chains would be to assign the result of each method call to a new variable, and progressively call the methods on each intermediate RDD.

By the way, Spark uses a “lazy” approach to generating RDDs, similar to PETL (Lab 4). It usually does no computation on data (including the generation of intermediate RDDs) until some results are actually requested, e.g. by calling `take` or `collect`.

To understand how the code above works, you can try a simpler example from http://www.mccarroll.net/blog/pyspark2/ (“Revisiting the wordcount example”). We have reproduced the code from that web page below, starting from the second shaded box onwards (note that the code for removing punctuation is different from the improved version above). Read the description of each step and then try running it.

**Code from the second shaded box:**

In [ ]:
lines = sc.parallelize(['Its fun to have fun,','but you have to know how.']) 
wordcounts = lines.map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower()) \
    .flatMap(lambda x: x.split()) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda x,y:x+y) \
    .map(lambda x:(x[1],x[0])) \
    .sortByKey(False) 
wordcounts.take(10)

**Code for step 1: `map(<function>)`**

In [ ]:
r1 = lines.map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower())
r1.take(10)

**Code for step 2: `flatMap(<function>)`**

In [ ]:
r2 = r1.flatMap(lambda x: x.split())
r2.take(20)

**Code for step 3: `map(<function>)` (again)**

In [ ]:
r3 = r2.map(lambda x: (x, 1))
r3.take(20)

**Code for step 4: `reduceByKey(<function>)`**

In [ ]:
r4 = r3.reduceByKey(lambda x,y:x+y)
r4.take(20)

**Code for step 5: `map(<function>)` (yet again!)**

In [ ]:
r5 = r4.map(lambda x:(x[1],x[0]))
r5.take(20)

**Code for step 6: `sortByKey( ascending=True|False )`**

In [ ]:
r6 = r5.sortByKey(ascending=False)
r6.take(20)

Now, leaving that example, make sure that you understand the difference between the functions `map` and `flatMap`, by predicting what each of the following three cells will do, and then running them to check whether your prediction is correct.

In [ ]:
words = sc.parallelize(["aardvark", "baby", "carrot"]) # Create an RDD from a Python list
word_chars = words.map(list)  # The function 'list' converts a string to a list of characters (which are just short strings)
word_chars.collect() # Collect gathers the RDD into a Python list.

In [ ]:
all_chars = words.flatMap(list)
all_chars.collect()

In [ ]:
distinct_chars = all_chars.distinct()
distinct_chars.collect()

## Exercise 2: Exploring the Enron dataset

Next, you will explore a dataset of email messages from the Enron corporation. You can read about it [here](https://en.wikipedia.org/wiki/Enron_Corpus).

First, remove the Exercise 1 dataset from memory. The output should be 0, indicating the success of the garbage collection.

In [ ]:
%reset -sf
import gc
gc.collect()

Rather than reading the data into a Spark resilient distributed dataset (RDD), we will  create a Spark *data frame* from the JSON data file. A data frame includes schema information (inferred from the JSON file in this case).

The `printSchema()` method call below prints the schema, and the `first()` method call prints the first message in the data set. The variable `spark` is predefined by PySpark to hold a reference to a *Spark Session* object, which is used instead of the Spark context to create a data frame.

This code will probably while take a while to run.

In [ ]:
messages = spark.read.json('/mnt/t-drive/InfoSci/INFO408/Spark/messages.json')
messages.createOrReplaceTempView("messages") # This is needed before doing any subsequent SQL queries on this data frame
print messages, '\n'
messages.printSchema() # Print the schema
print 'Number of messages:', messages.count()
messages.first() # Look at the first email message

Spark data frames support the use of SQL (extended to allow nested column/property names, e.g. `headers.From`).

In [ ]:
# Find the total number of message senders in the data set
spark.sql("select count(distinct headers.From) fromcount from messages").collect()

The result is expressed as a DataFrame that comprises a single row.

There are also a range of useful methods we can use on our data frame, for example:

In [ ]:
# Count emails from louis.soldano@...
messages.filter(messages.headers.From == 'louis.soldano@enron.com').count()

## Assessment

Answer the questions below by writing your Python code in the corresponding cells below. Save the modified notebook and upload it to Blackboard. You do not need to provide the output from the code. The deadline to submit your answers is **Thursday 1 October at 5pm.**

Explore the use of SQL queries and/or dataframe and RDD methods to summarise and extract information from the Enron dataset. The following web page might be useful to see what operations can be performed on data frames and RDDs: https://github.com/lgallen/pyspark_dataframes/blob/master/housing_model.ipynb (section “Dataframe operations”).

Documentation for the methods that can be called on data frames: https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.DataFrame

Documentation for the methods that can be called on RDDs: http://spark.apache.org/docs/2.4.3/api/python/pyspark.html#pyspark.RDD

Some built-in Spark functions: http://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#module-pyspark.sql.functions

Once you have tried some basic queries, try to answer the questions below. It is OK to report on partially successful attempts! You can add commentary in code comments or additional text cells. Also feel free to invent your own questions about this dataset to answer using Spark.

<ol>
    <li>
        <p><strong>What is the distribution of message body word counts across all messages?</strong> In other words, create an RDD or data frame that maps word counts (how many words are in a message body) to the number of messages with that word count. Rather than giving exact word counts, round them to the closest power of 10 (you can use <code>round(count, -1)</code>). Note that <code>messages</code> is a data frame, whereas this problem might be easily solved using the methods that can be called on an RDD. However, you can extract an RDD from the data frame, e.g:</p>
    </li>
</ol>

In [ ]:
messagesRdd = messages.rdd

<ol>
    <li style="list-style: none;">
        <p>The new RDD will contain <code>Row</code> objects. If you call <code>map()</code> on <code>messagesRdd</code> as defined above, the function you provide to <code>map</code> will need to take a row object as its parameter, e.g. <code>messagesRdd.map(lambda row: ...)</code>. See the <a href="https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.Row" target="_blank">Row class documentation</a> to find out how to access the columns of a row (basically, it is the same as for a data frame).</p>
    </li>
</ol>

In [ ]:
# Code to answer question 1.


<ol start="2">
    <li>
        <p><strong>Which words appear most commonly within message subject lines?</strong></p>
        <p><strong>Hint:</strong> this is very similar to the standard map-reduce example of counting words in documents — see the word count example in Exercise 1. The data processed in that example is an RDD, whereas we have a data frame, so you may want to use <code>messagesRdd</code> as defined above.</p>
        <p><strong>Tip for sorting an RDD:</strong> To sort an RDD containing key-value pairs (represented in Python as <code>(key, value)</code> tuples), you can use the RDD method <code>sortBy()</code>. To sort by <em>value</em> in descending order, call <code>sortBy(lambda pair: pair[1], ascending=False)</code>.</p>
    </li>
</ol>

In [ ]:
# Code to answer question 2.


<ol start="3">
    <li><strong>Which senders sent the most messages?</strong></li>
</ol>

In [ ]:
# Code to answer question 3.


## Challenge question (not assessed)

<p><strong>What are the unique sender/recipient pairs together with the number of times each pair occurs?</strong></p>
<p><strong>Note:</strong> This query requires parsing the <code>headers.To</code> string to find the individual recipients, as in general an email message may have more than one recipient, separated by commas followed by a space, tab or newline character. Extracting individual recipients can be done using map-reduce on the RDD underlying the data frame. We can also do this using the <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/select.html#lateral-view" target="_blank">Spark SQL <em>lateral view</em> construct</a> and the <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.explode" target="_blank"><code>explode</code></a> and <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.split" target="_blank"><code>split</code></a> functions.</p>
<p>The following query creates a new data frame with columns named <code>sender</code> and <code>recipient</code>. There is a row for each occurrence of a sender-recipient pair in the data set (there may be multiple entries for any given pair). (Note that the column names <code>from</code> and <code>to</code> have been changed to <code>sender</code> and <code>recipient</code> to avoid confusing Spark SQL, which does not like a column named <code>from</code>.)</p>

In [ ]:
sender_recipient_pairs = spark.sql("select messages.headers.From as sender, recipient, count(*) as count from messages lateral view explode(split(messages.headers.To, '[,\\\\s]+')) tolist as recipient group by sender, recipient order by count desc")
sender_recipient_pairs.createOrReplaceTempView("sender_recipient_pairs")
sender_recipient_pairs.printSchema()
sender_recipient_pairs.take(5)

The next cell produces a new data frame that excludes the emails sent or CCed to self.

In [ ]:
no_self_mails_sender_recipient_pairs = spark.sql("select sender, recipient, count from sender_recipient_pairs where sender <> recipient")
no_self_mails_sender_recipient_pairs.createOrReplaceTempView("no_self_mails_sender_recipient_pairs")
no_self_mails_sender_recipient_pairs.take(5)

We have just shown you how to answer this question using data frames, so try to answer this question using a different technique (e.g., convert the dataframe to an RDD and use RDD methods).

In [ ]:
# Code to answer the challenge question.
